In [2]:
import sys
sys.path.append(r'C:\Users\kmm171\ImageAnalysis')
from pathlib import Path
import os
import shutil
from tqdm import tqdm

#local imports
import main.main_legacy as legacy
import main.main_improv2 as improv
import main.main as main
from caImageAnalysis.utilities.pathutils import pathcrawler, moveoriginalimage, renametxtfile



In [8]:
basefolder = r'C:\Users\kmm171\Desktop\Data\gut_tectum\20240205_7dpf'

fish1 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish2 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish3 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish4 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\142_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125'
fish5 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\143_eval3h2bgcamp6s_7dpf_unfed_mono_R_20240125'
fish6 = r'C:\Users\kmm171\Desktop\Data\improv\training_data\20240125_7dpf\144_eval3h2bgcamp6s_7dpf_unfed_mono_R_20240125'

fishpaths = {

    "fish" + "139_a" : pathcrawler(fish1, set(), [], "img_stack_0"),
    "fish" + "139_b" : pathcrawler(fish1, set(), [], "img_stack_1"),
    "fish" + "140b_a" : pathcrawler(fish2, set(), [], "img_stack_0"),
    "fish" + "140b_b" : pathcrawler(fish2, set(), [], "img_stack_1"),
    "fish" + "141_a" : pathcrawler(fish3, set(), [], "img_stack_0"),
    "fish" + "141_b" : pathcrawler(fish3, set(), [], "img_stack_1"),
    "fish" + "142_a" : pathcrawler(fish4, set(), [], "img_stack_0"),
    "fish" + "142_b" : pathcrawler(fish4, set(), [], "img_stack_1"),
    "fish" + "143_a" : pathcrawler(fish5, set(), [], "img_stack_0"),
    "fish" + "143_b" : pathcrawler(fish5, set(), [], "img_stack_1"),
    "fish" + "144_a" : pathcrawler(fish6, set(), [], "img_stack_0"),
    "fish" + "144_b" : pathcrawler(fish6, set(), [], "img_stack_1"),

}
renametxtfile(fishpaths)
print(fishpaths)

{'fish139_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'], 'fish139_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\139_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'], 'fish140b_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'], 'fish140b_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\140b_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'], 'fish141_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_0'], 'fish141_b': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\141_eval3h2bgcamp6s_7dpf_fed_mono_R_20240125\\img_stack_1'], 'fish142_a': ['C:\\Users\\kmm171\\Desktop\\Data\\improv\\training_data\\20240125_7dpf\\142_eval3h2bgcamp6s_

In [9]:
volume = False
old = False
input_tau = 1.5

In [10]:
#Create a class for each fish
fishSchool = {}
if not volume: #old means that I used the old json file and so the dictionary of stimulus from constants is different.
    for fish, paths in fishpaths.items():
        for path in paths:
            dory = improv.Fish(path, stimkey="output", old_stims=old)
            fishSchool[f'{fish}'] = dory

    #single movement correction
    for key, fish in tqdm(fishSchool.items(), "Movement correction of fish"):
        fish.singleMoveCorrection()

elif volume:
    #create a class for each fish
    for fish, paths in fishpaths.items():
        for path in paths:
            volfish = improv.Fish(path, stimkey="output")
            fishSchool[f'{fish}'] = volfish

    #split volumes
    for fish, volfish in fishSchool.items():
        if len(volfish.dataPaths['volumes']) == 0:
            volfish.legacy_volumesplit(force=True, crop=False)
        else:
            print("found splitted volumes")

    #copy some files into each img_stack folder
    for fish, datapath in fishpaths.items():
        datapath = datapath[0]
        basePath = Path(datapath)
        fish_paths = []

        with os.scandir(Path(datapath)) as entries:
            for entry in entries:
                if os.path.isdir(entry.path) and 'img_stack' in entry.name:
                    fish_paths.append(entry.path)

        for file_name in os.listdir(Path(datapath)):
            if 'log' in file_name and file_name.endswith(".txt"):
                log_source = datapath + "\\" + file_name
                if os.path.isfile(log_source) and os.path.isfile(log_source):
                    for destination in tqdm(fish_paths, desc="log files successfully copied"):
                        shutil.copy(log_source, destination)

        for file_name in os.listdir(Path(datapath)):
            if 'output' in file_name and file_name.endswith(".txt"):
                output_source = datapath + "\\" + file_name
                if os.path.isfile(output_source) and os.path.isfile(output_source):
                    for destination in tqdm(fish_paths, desc="stim_output files successfully copied"):
                        shutil.copy(output_source, destination)

        for file_name in os.listdir(Path(datapath)):
            if 'stim_details' in file_name and file_name.endswith(".txt"):
                output_source = datapath + "\\" + file_name
                if os.path.isfile(output_source) and os.path.isfile(output_source):
                    for destination in tqdm(fish_paths, desc="stim_output files successfully copied"):
                        shutil.copy(output_source, destination)

    #volume movement correction
    for fish, volfish in fishSchool.items():
        volfish.volumeMoveCorrection()

    #moves original image into the original_image folder after movement correction
    for key, fishes in tqdm(fishpaths.items(), "Moving original image of fish"):
        for fish in fishes:
            img_stack_path = pathcrawler(f"{fish}", set(), [], "img_stack")
            for stack in img_stack_path:
                moveoriginalimage(stack)
    '''
    else:
        for fish, paths in fishpaths.items():
            for path in paths:
                dory = legacy.Fish(path, stimkey="output")
                fishSchool[f'{fish}'] = dory

        single movement correction
        for key, fish in tqdm(fishSchool.items(), "Movement correction of fish"):
            fish.singleMoveCorrection()
    '''

Movement correction of fish: 100%|██████████| 12/12 [00:00<00:00, 2174.25it/s]

dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it
dory found it


In [ ]:
#move original image file to the original_image folder.
for key, fishes in tqdm(fishpaths.items(), "Moving original image of fish"):
    for fish in fishes:
        moveoriginalimage(fish)

## Before proceeding, create masks

## Cell identification - Suite2P

In [11]:
if volume:
    for fish, volfish in fishSchool.items():
        volfish.volumeSuite2p(input_tau=input_tau, force=True)
else:
    for fish, volfish in tqdm(fishSchool.items(), "cell detection of fish"):
            volfish.singleSuite2p(input_tau=input_tau)

cell detection of fish:   0%|          | 0/12 [00:00<?, ?it/s]

{}
tif
** Found 1 tifs - converting to binary **
time 2.95 sec. Wrote 748 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: applying default C:\Users\kmm171\.suite2p\classifiers\classifier_user.npy
----------- REGISTRATION
Reference frame, 8.49 sec.
Registered 500/748 in 321.52s
Registered 748/748 in 494.02s


C:\Users\kmm171\AppData\Roaming\Python\Python39\site-packages\suite2p\registration\register.py:45: RuntimeWarning: invalid value encountered in divide
  dxy = dxy / dxy.mean()


----------- Total 510.37 sec
----------- ROI DETECTION
Binning movie in chunks of length 01
Binned movie of size [748,392,392] created in 0.60 sec.


C:\Users\kmm171\AppData\Roaming\Python\Python39\site-packages\suite2p\detection\sparsedetect.py:270: UserWarning: Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.
  warn("Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.")


...

C:\Users\kmm171\AppData\Roaming\Python\Python39\site-packages\suite2p\registration\register.py:45: RuntimeWarning: invalid value encountered in divide
  dxy = dxy / dxy.mean()


----------- Total 656.80 sec
----------- ROI DETECTION
Binning movie in chunks of length 01
Binned movie of size [749,392,392] created in 0.60 sec.


C:\Users\kmm171\AppData\Roaming\Python\Python39\site-packages\suite2p\detection\sparsedetect.py:270: UserWarning: Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.
  warn("Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.")


NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 5.00 
0 ROIs, score=44.75
1000 ROIs, score=21.76
2000 ROIs, score=18.90
3000 ROIs, score=16.79
4000 ROIs, score=15.10
Detected 5000 ROIs, 23.00 sec
['npix_norm', 'compact']
Preclassify threshold 0.15, 1725 ROIs removed
After removing overlaps, 3103 ROIs remain
----------- Total 28.39 sec.
----------- EXTRACTION
Masks created, 2.51 sec.
Extracted fluorescence from 3103 ROIs in 749 frames, 1.93 sec.
----------- Total 4.71 sec.
----------- CLASSIFICATION
['npix_norm', 'skew', 'compact']
----------- SPIKE DECONVOLUTION
----------- Total 0.16 sec.
Plane 0 processed in 690.32 sec (can open in GUI).
total = 692.93 sec.
TOTAL RUNTIME 692.93 sec


cell detection of fish: 100%|██████████| 12/12 [2:15:11<00:00, 675.96s/it]


In [7]:
%store volume old

Stored 'volume' (bool)
Stored 'old' (bool)
